In [125]:
import networkx as nx
import random 
import torch
import math

In [126]:
G = nx.Graph()

nx.add_path(G, [0,1,3,5])
nx.add_path(G, [1,4,5])
nx.add_path(G, [0,2,3])
nx.add_path(G, [0,2,4])
nx.add_path(G, [0,1,4])
nx.add_path(G, [0,6,7, 8])

In [127]:
def random_neighbours(l):
    a = list(l)
    random.shuffle(a)
    return iter(a)

In [128]:
k2 = 2
k1 = 1
num_samples = 3

In [129]:
distance_graphs = []
distance_graphs_attr_ids = [] # k2* [num_rand_sample, num_edges, d]
for _ in range(k2):
    distance_graphs.append(nx.Graph())

In [130]:
edge_attr = torch.tensor([[0, 10], [0,20], [1, 2], [4,5], [5,6], [7,8], [9,10], [100, 200], [300, 400], [100, 200], [300, 400]]) # torch.size(num_edge, feature_size)
edge_attr.shape

torch.Size([11, 2])

In [131]:
edge_ids = {}
for i, e in enumerate(G.edges()):
    edge_ids[e] = i
    edge_ids[(e[1], e[0])] = i #only for test


In [132]:
for i in range(k2): 
    distance_graphs_attr_ids.append([])
    for s in range(num_samples):
        distance_graphs_attr_ids[i].append([])


for v in range(G.number_of_nodes()):
    
    dist_layes = None
    for s in range(num_samples):
        if s < 1: #compute only on first sample
            dist_layes = dict(enumerate(nx.bfs_layers(G, [v])))

        bsf_tree = nx.reverse(nx.bfs_tree(G, source=v, depth_limit=None, sort_neighbors=random_neighbours))
        
        
        for i in range(k2):
            d = 2*math.pow(3, i)*k1
                    
            if not d in dist_layes.keys(): #new
                continue

            for u in dist_layes[d]:
                if s < 1: #save only on first sample
                    distance_graphs[i].add_edge(v, u)

                edges_on_u_v_path = list(nx.dfs_edges(bsf_tree, source=u))
                
                edge_feature_id = []
                for e in edges_on_u_v_path:
                    edge_feature_id.append(edge_ids[e])
                distance_graphs_attr_ids[i][s].append(edge_feature_id)


In [133]:
for i in range(k2):
    idx = torch.tensor(distance_graphs_attr_ids[i])
 
    if idx.shape[1] == 0: #not path for layer i
        pass 
    else:
        distance_graph_attr = edge_attr[idx]
